
We are using Google Colab due to the GPUs and the quick runtime.

Code is largely based off of the following source: https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/vie.txt'
df = pd.read_csv(path, header= None, sep='\t')

In [ ]:
df=df.drop(columns=2)

In [ ]:
df

,0,1
0,Run!,Chạy!
1,Help!,Giúp tôi với!
2,Go on.,Tiếp tục đi.
3,Hello!,Chào bạn.
4,Hurry!,Nhanh lên nào!
...,...,...
6364,"In 2009, Selena Gomez became the youngest pers...","Vào năm 2009, Sê-lê-na Gô-mét đã được lựa chọn..."
6365,"In 2009, Selena Gomez became the youngest pers...","Vào năm 2009, Selena Gomez đã được lựa chọn để..."
6366,"In 2009, Selena Gomez became the youngest pers...","Vào năm 2009, Selena Gomez đã trở thành Đại sứ..."
6367,The people here are particular about what they...,Những người ở đây khá là khó tính về khẩu vị ă...


This dataset has many duplicates (English -> Vietnamese). While it might be beneficial for a person studying a language to have these variations of phrases, due to the simplicity of our model, we chose to remove these duplicates.

In [ ]:
df_simple = df.drop_duplicates([0])
df_simple

,0,1
0,Run!,Chạy!
1,Help!,Giúp tôi với!
2,Go on.,Tiếp tục đi.
3,Hello!,Chào bạn.
4,Hurry!,Nhanh lên nào!
...,...,...
6361,"The people crowded round the injured man, but ...","Nhiều người đã vây xung quanh người bị thương,..."
6362,It's very easy to sound natural in your own na...,Câu nói đó trong ngôn ngữ của bạn rất dễ để ng...
6363,"In 2009, Selena Gomez became the youngest pers...","Vào năm 2009, Selena Gomez đã được lựa chọn để..."
6367,The people here are particular about what they...,Những người ở đây khá là khó tính về khẩu vị ă...


In [ ]:
df_simple.to_csv(r'/content/drive/MyDrive/vie_simple.txt', header=None, index=None, sep='\t', mode='a')

In [ ]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# normalize unicode characters
			line = normalize('NFD', line).encode('ascii', 'ignore')
			line = line.decode('UTF-8')
			# tokenize on white space
			line = line.split()
			# convert to lowercase
			line = [word.lower() for word in line]
			# remove punctuation from each token
			line = [word.translate(table) for word in line]
			# remove non-printable chars form each token
			line = [re_print.sub('', w) for w in line]
			# remove tokens with numbers in them
			line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

In [ ]:
# load dataset
filename = '/content/drive/MyDrive/vie_simple.txt'
doc = load_doc(filename)
# split into english-vietnamese pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'english-vietnamese.pkl')
# spot check
for i in range(100):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: english-vietnamese.pkl
[run] => [chay]
[help] => [giup toi voi]
[go on] => [tiep tuc i]
[hello] => [chao ban]
[hurry] => [nhanh len nao]
[eat it] => [an i]
[help me] => [cuu toi voi]
[i agree] => [toi cung nghi nhu vay]
[perfect] => [hoan hao]
[we know] => [chung toi biet]
[you run] => [ban chay]
[cheer up] => [ung co rau ri qua nhu the]
[he tries] => [han thu]
[hurry up] => [thoang cai chan len]
[i forgot] => [toi quen mat roi]
[im bald] => [toi bi hoi]
[im busy] => [toi ang ban]
[too late] => [muon qua]
[i hate tv] => [toi ghet ti vi]
[i laughed] => [toi a cuoi]
[i will go] => [toi se i]
[is it bad] => [no co toi khong]
[its cold] => [lanh]
[its ours] => [o la cua chung toi]
[she cried] => [no a khoc]
[sit there] => [hay ngoi o o]
[whats up] => [gi the]
[are you ok] => [ban co sao khong]
[find a job] => [hay tim mot cong viec i]
[hurry home] => [hay mau ve nha i]
[i can read] => [toi oc uoc]
[i guess so] => [toi oan vay]
[i hate you] => [toi ghet anh]
[i love you] => [anh phai

In [ ]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-vietnamese.pkl')
# reduce dataset size
n_sentences = 1000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:900], dataset[900:]
# save
save_clean_data(dataset, 'english-vietnamese-both.pkl')
save_clean_data(train, 'english-vietnamese-train.pkl')
save_clean_data(test, 'english-vietnamese-test.pkl')


Saved: english-vietnamese-both.pkl
Saved: english-vietnamese-train.pkl
Saved: english-vietnamese-test.pkl


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras import optimizers

In [ ]:
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# load datasets
dataset = load_clean_sentences('english-vietnamese-both.pkl')
train = load_clean_sentences('english-vietnamese-train.pkl')
test = load_clean_sentences('english-vietnamese-test.pkl')

In [ ]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
  
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [ ]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

# prepare vietnamese tokenizer
vie_tokenizer = create_tokenizer(dataset[:, 1])
vie_vocab_size = len(vie_tokenizer.word_index) + 1
vie_length = max_length(dataset[:, 1])
print('Vietnamese Vocabulary Size: %d' % vie_vocab_size)
print('Vietnamese Max Length: %d' % (vie_length))

English Vocabulary Size: 916
English Max Length: 6
Vietnamese Vocabulary Size: 552
Vietnamese Max Length: 11


We added one hot encoding to create a binary for our categorical variables. Doing this will allow the model to run properly as categorical variables cannot be used in most machine learning models.

In [ ]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [ ]:
# prepare training data
trainX = encode_sequences(vie_tokenizer, vie_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(vie_tokenizer, vie_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

We chose to utilize the Sequential Sequence2Sequence model from keras in order to give an output to a sequence of words given a sequence of words. Seq2Seq runs based on encoders and decoders and can automatically learn mapped sentence pairs in each language in order to predict an output sentence. We chose this model because it works best with our data and completes the job we need - translating one language to another.

Bidirectional LSTM was used to gain a better performance of our model. Bidirectional LSTM trains on two LSTM layers rather than one, giving the model better results overall.

In [ ]:
from keras.layers.wrappers import Bidirectional

# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(Bidirectional(LSTM(n_units)))
	model.add(RepeatVector(tar_timesteps))
	model.add(Bidirectional(LSTM(n_units, return_sequences=True)))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

# define model
model = define_model(vie_vocab_size, eng_vocab_size, vie_length, eng_length, 256)
#rms = optimizers.RMSprop(lr=0.0001)
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

#optimizer=rms

# summarize defined model
print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 11, 256)           141312    
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               1050624   
_________________________________________________________________
repeat_vector (RepeatVector) (None, 6, 512)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 6, 512)            1574912   
_________________________________________________________________
time_distributed (TimeDistri (None, 6, 916)            469908    
Total params: 3,236,756
Trainable params: 3,236,756
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
filename = 'model.h6'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Epoch 1/30
15/15 - 42s - loss: 6.0459 - accuracy: 0.3372 - val_loss: 4.3907 - val_accuracy: 0.3700

Epoch 00001: val_loss improved from inf to 4.39072, saving model to model.h6


INFO:tensorflow:Assets written to: model.h6/assets


INFO:tensorflow:Assets written to: model.h6/assets


Epoch 2/30
15/15 - 0s - loss: 4.1057 - accuracy: 0.3917 - val_loss: 4.0745 - val_accuracy: 0.4100

Epoch 00002: val_loss improved from 4.39072 to 4.07452, saving model to model.h6


INFO:tensorflow:Assets written to: model.h6/assets


...

INFO:tensorflow:Assets written to: model.h6/assets


Epoch 12/30
15/15 - 0s - loss: 3.2704 - accuracy: 0.4452 - val_loss: 3.8958 - val_accuracy: 0.4533

Epoch 00012: val_loss did not improve from 3.88021
Epoch 13/30
15/15 - 0s - loss: 3.1661 - accuracy: 0.4524 - val_loss: 3.8676 - val_accuracy: 0.4550

Epoch 00013: val_loss improved from 3.88021 to 3.86761, saving model to model.h6


INFO:tensorflow:Assets written to: model.h6/assets


INFO:tensorflow:Assets written to: model.h6/assets


Epoch 14/30
15/15 - 0s - loss: 3.0737 - accuracy: 0.4602 - val_loss: 3.8781 - val_accuracy: 0.4583

Epoch 00014: val_loss did not improve from 3.86761
Epoch 15/30
15/15 - 0s - loss: 2.9934 - accuracy: 0.4689 - val_loss: 3.9755 - val_accuracy: 0.4583

Epoch 00015: val_loss did not improve from 3.86761
Epoch 16/30
15/15 - 0s - loss: 2.8926 - accuracy: 0.4767 - val_loss: 3.9236 - val_accuracy: 0.4683

Epoch 00016: val_loss did not improve from 3.86761
Epoch 17/30
15/15 - 0s - loss: 2.7985 - accuracy: 0.4817 - val_loss: 3.9756 - val_accuracy: 0.4633

Epoch 00017: val_loss did not improve from 3.86761
Epoch 18/30
15/15 - 0s - loss: 2.6988 - accuracy: 0.4831 - val_loss: 4.0073 - val_accuracy: 0.4467

Epoch 00018: val_loss did not improve from 3.86761
Epoch 19/30
15/15 - 0s - loss: 2.6055 - accuracy: 0.4881 - val_loss: 4.0232 - val_accuracy: 0.4667

Epoch 00019: val_loss did not improve from 3.86761
Epoch 20/30
15/15 - 0s - loss: 2.4863 - accuracy: 0.4989 - val_loss: 4.0892 - val_accuracy: 0.

In [ ]:
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load model
model = load_model('model.h6')

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
#Attempt 3 of 5/8/21 (bidirectional lstm layers)

# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

train
src=[toi thich anh nen], target=[i like candlelight], predicted=[i i]
src=[khong co gioi han], target=[there are no limits], predicted=[is is]
src=[to tung bi bat coc], target=[i was kidnapped], predicted=[i i]
src=[ban co the lam uoc], target=[you can do it], predicted=[you you you]
src=[toi can co mot may vi tinh moi], target=[i need a new computer], predicted=[i i to to]
src=[tai sao cau lai hon tom], target=[why did you kiss tom], predicted=[tom tom tom a]
src=[tiep tuc i], target=[go on], predicted=[is is]
src=[ang ua ay a], target=[youre joking], predicted=[is is]
src=[ai a lam chiec banh nay], target=[who made this cake], predicted=[is is]
src=[bon minh can nghi lon hon], target=[we need to think big], predicted=[i i a a]


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.169236
BLEU-2: 0.020689
BLEU-3: 0.087374
BLEU-4: 0.125255
test
src=[ho a i cau ca], target=[they went fishing], predicted=[you is a]
src=[chung ta khong the bo cuoc], target=[we cant give up], predicted=[its is a]
src=[ho bat toi], target=[they arrested me], predicted=[i to]
src=[rat hiem loi phan nan], target=[complaints are rare], predicted=[its is]
src=[ban co the vao], target=[you may enter], predicted=[you you you]
src=[thu nay lam viec tot qua], target=[this is working great], predicted=[is is a]
src=[hai nguoi o lai ay], target=[you two stay here], predicted=[is is]
src=[khong co gioi han], target=[there is no limit], predicted=[is is]
src=[tat ca moi nguoi eu ghet tom], target=[everybody hates tom], predicted=[tom tom a]
src=[toi bi bat a], target=[am i under arrest], predicted=[i i]
BLEU-1: 0.142855
BLEU-2: 0.023481
BLEU-3: 0.093598
BLEU-4: 0.132252
